# Get Started With nGQL Jupyter Magic Extension

In [ ]:
%pip install ipython-ngql

> We need to have a running NebulaGraph Cluster to test this.

On Colab, we could leverage [NebulaGraph-Lite](https://github.com/wey-gu/nebulagraph-lite/) to do so, for more options please refer to [NebulaGraph Docs](https://docs.nebula-graph.io).

In [ ]:
%pip install nebulagraph-lite

In [3]:
from nebulagraph_lite import nebulagraph_let as ng_let

n = ng_let()

# This takes around 5 mins
n.start()

Streaming output truncated to the last 5000 lines.
usr/share/i18n/locales/ja_JP
usr/share/i18n/locales/ka_GE
usr/share/i18n/locales/kk_KZ
usr/share/i18n/locales/kl_GL
usr/share/i18n/locales/km_KH
usr/share/i18n/locales/kn_IN
usr/share/i18n/locales/ko_KR
usr/share/i18n/locales/kok_IN
usr/share/i18n/locales/ks_IN
usr/share/i18n/locales/ks_IN@devanagari
usr/share/i18n/locales/ku_TR
usr/share/i18n/locales/kw_GB
usr/share/i18n/locales/ky_KG
usr/share/i18n/locales/lb_LU
usr/share/i18n/locales/lg_UG
usr/share/i18n/locales/li_BE
usr/share/i18n/locales/li_NL
usr/share/i18n/locales/lij_IT
usr/share/i18n/locales/lo_LA
usr/share/i18n/locales/lt_LT
usr/share/i18n/locales/lv_LV
usr/share/i18n/locales/mag_IN
usr/share/i18n/locales/mai_IN
usr/share/i18n/locales/mg_MG
usr/share/i18n/locales/mhr_RU
usr/share/i18n/locales/mi_NZ
usr/share/i18n/locales/mk_MK
usr/share/i18n/locales/ml_IN
usr/share/i18n/locales/mn_MN
usr/share/i18n/locales/mni_IN
usr/share/i18n/locales/mr_IN
usr/share/i18n/locales/ms_MY
usr/

## How To Run nGQL in Jupyter or iPython
First, install nGQL Magic with pip:
```bash
!pip install ipython-ngql
```
Second, load extension:
```bash
$load_ext ngql
```

In [2]:
%load_ext ngql

## How to Use
Connect to Nebula Graph with:

```bash
%ngql --address <ip> --port <port> --user <username> --password <password>
```

In [5]:
%ngql --address 127.0.0.1 --port 9669 --user root --password nebula

Connection Pool Created


,Name
0,basketballplayer


Make a query after `Connection Pool Created` shown in the connection from last step:


Option 1, it supports one line query as:
```ngql
%ngql <query_line>;
```
Option 2, it also supports multiple queries:

```ngql
%%ngql
<line 0>;
<line 1>;
```

> Note, we will support a 3rd option to enable runing lines from `.ngql` file soon.

In [4]:
%ngql USE basketballplayer;
%ngql MATCH (v:player{name:"Tim Duncan"})-->(v2:player) RETURN v2.player.name AS Name;

,Name
0,Tony Parker
1,Manu Ginobili


In [5]:
%%ngql
SHOW TAGS;
SHOW HOSTS;

,Host,Port,Status,Leader count,Leader distribution,Partition distribution,Version
0,storaged0,9779,ONLINE,211,"basketballplayer:3, csci_demo:33, customer_ser...","basketballplayer:3, csci_demo:33, customer_ser...",3.6.0
1,storaged1,9779,ONLINE,210,"basketballplayer:3, csci_demo:33, customer_ser...","basketballplayer:3, csci_demo:33, customer_ser...",3.6.0
2,storaged2,9779,ONLINE,229,"basketballplayer:4, chinese_kg:1, coe:1, csci_...","basketballplayer:4, chinese_kg:1, coe:1, csci_...",3.6.0


> It's quite easy, right?
You could get help by `%ngql help` for some advanced usage:

```ngql
%ngql help
```

### Using Variables in Query String

We used Jinja2(https://jinja.palletsprojects.com/) as templating method for variables in query string:

```python
trainer = "Sue"
```

```ngql
%%ngql
GO FROM "{{ trainer }}" OVER owns_pokemon YIELD owns_pokemon._dst as pokemon_id | GO FROM $-.pokemon_id OVER owns_pokemon REVERSELY YIELD owns_pokemon._dst AS Trainer_Name;
```

In [6]:
vid = "player100"

In [7]:
%%ngql
MATCH (v)<-[e:follow]- (v2)-[e2:serve]->(v3)
  WHERE id(v) == "{{ vid }}"
RETURN v2.player.name AS FriendOf, v3.team.name AS Team LIMIT 3;

,FriendOf,Team
0,Dejounte Murray,Spurs
1,Shaquille O'Neal,Lakers
2,Shaquille O'Neal,Suns


### Using Raw thrift data type as result
By default the result `ngql_result_style` is `pandas`, this enabled us to have table view renderred by Jupyter Notebook.

While, if you would like to get raw results from `neutron2-python` itself, just configure it as below on the fly:

```
%config IPythonNGQL.ngql_result_style="raw"
```

And after querying, the result will be stored in `_`, plesae then refer it to a new variable for further ad-hoc tweaking on it like:
```
$ngql <query>;

result = _

dir(result)
```

In [10]:
%config IPythonNGQL.ngql_result_style="raw"

In [11]:
%%ngql USE basketballplayer;
GO 2 STEPS FROM "player102" OVER follow YIELD dst(edge);

ResultSet(keys: ['dst(EDGE)'], values: ["player100"],["player102"],["player125"],["player101"],["player125"])

In [12]:
r = _

In [13]:
r.column_values("dst(EDGE)")[0].cast()

'player100'

> Change back to `pandas` `ngql_result_style`

In [14]:
%config IPythonNGQL.ngql_result_style="pandas"

In [15]:
%%ngql
GO FROM "player100", "player102" OVER serve \
  WHERE properties(edge).start_year > 1995 \
YIELD DISTINCT properties($$).name AS team_name, properties(edge).start_year AS start_year, properties($^).name AS player_name;

,team_name,start_year,player_name
0,Spurs,1997,Tim Duncan
1,Trail Blazers,2006,LaMarcus Aldridge
2,Spurs,2015,LaMarcus Aldridge


# Draw nGQL queries

> This depends on pyvis

In [ ]:
%pip install pyvis

In [8]:
%ngql match p=(:player)-[]->() return p LIMIT 5

,p
0,"(""player147"" :player{age: 36, name: ""Amar'e St..."
1,"(""player147"" :player{age: 36, name: ""Amar'e St..."
2,"(""player147"" :player{age: 36, name: ""Amar'e St..."
3,"(""player147"" :player{age: 36, name: ""Amar'e St..."
4,"(""player146"" :player{age: 33, name: ""Dwight Ho..."


In [9]:
%ng_draw

<class 'pyvis.network.Network'> |N|=7 |E|=5

In [10]:
%ngql GET SUBGRAPH 2 STEPS FROM "player101" YIELD VERTICES AS nodes, EDGES AS relationships;

,nodes,relationships
0,"[(""player101"" :player{})]","[(""player101"")-[:serve@0{}]->(""team204""), (""pl..."
1,"[(""team215"" :team{}), (""player113"" :player{}),...","[(""player111"")-[:serve@0{}]->(""team215""), (""pl..."
2,"[(""player146"" :player{}), (""team206"" :team{}),...","[(""player146"")-[:serve@0{}]->(""team222""), (""pl..."


In [11]:
%ng_draw

<class 'pyvis.network.Network'> |N|=36 |E|=86

# Draw Graph Schema

We could quickly draw the schema with `%ng_draw_schema`

In [3]:
%ngql USE demo_supplychain

""


In [4]:
%ng_draw_schema

<class 'pyvis.network.Network'> |N|=4 |E|=3

## Only one takeaway: `% ngql help`!
All you have to remember is to use `$ngql help` to have all hints :-)

In [21]:
%ngql help



        Supported Configurations:
        ------------------------
        
        > How to config ngql_result_style in "raw", "pandas"
        %config IPythonNGQL.ngql_result_style="raw"
        %config IPythonNGQL.ngql_result_style="pandas"

        > How to config ngql_verbose in True, False
        %config IPythonNGQL.ngql_verbose=True

        > How to config max_connection_pool_size
        %config IPythonNGQL.max_connection_pool_size=10

        Quick Start:
        -----------

        > Connect to Neubla Graph
        %ngql --address 127.0.0.1 --port 9669 --user user --password password

        > Use Space
        %ngql USE basketballplayer

        > Query
        %ngql SHOW TAGS;

        > Multile Queries
        %%ngql
        SHOW TAGS;
        SHOW HOSTS;

        Reload ngql Magic
        %reload_ext ngql

        > Variables in query, we are using Jinja2 here
        name = "nba"
        %ngql USE "{{ name }}"

        > Query and draw the graph

        %ngql GET 